<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/gradcam_cnn_vit_gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# full_script_with_gradcam.py — paste into one notebook cell
import os
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# sklearn split
from sklearn.model_selection import train_test_split

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# ------------------------
# SETTINGS
# ------------------------
dataset_dir = "/kaggle/input/alz-b-1100/alzheimer_new_11/alzheimer_new"  # update if needed
img_size = (128, 128)
expected_classes = 8  # set to the actual number of classes

# ------------------------
# LOAD DATA
# ------------------------
X = []
y = []

class_names = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
print("✅ Classes found:", class_names)

if len(class_names) != expected_classes:
    raise ValueError(f"Expected exactly {expected_classes} classes, found {len(class_names)}: {class_names}")

for idx, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_dir, class_name)
    print(f"🔍 Processing class '{class_name}' ({idx})")
    for root, _, files in os.walk(class_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img, dtype=np.float32) / 255.0  # normalize here
                X.append(img_array)
                y.append(idx)
            except Exception as e:
                print(f"⚠️ Failed to process {file_path}: {e}")

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

print(f"\n✅ Dataset loaded successfully:")
print(f"   X shape = {X.shape}")
print(f"   y shape = {y.shape}")
print(f"   Class mapping = {dict(enumerate(class_names))}")

# Save (optional)
np.save("X.npy", X)
np.save("y.npy", y)
print("✅ Saved X.npy and y.npy to current working directory.")

# ------------------------
# TRAIN/VAL SPLIT
# ------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# -----------------------
# MODEL (hybrid CNN + ViT + GRU)
# -----------------------
input_shape = (128, 128, 3)
patch_size = 16
num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
projection_dim = 64
transformer_layers = 4
num_heads = 8
num_classes = expected_classes

class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)

    def call(self, patches):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patches) + self.position_embedding(positions)
        return encoded

def build_hybrid_model():
    inputs = layers.Input(shape=input_shape)

    # DCNN branch
    x_cnn = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x_cnn = layers.MaxPooling2D((2,2))(x_cnn)
    x_cnn = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x_cnn)
    x_cnn = layers.MaxPooling2D((2,2))(x_cnn)
    x_cnn = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x_cnn)
    x_cnn = layers.MaxPooling2D((2,2))(x_cnn)
    x_cnn = layers.Flatten()(x_cnn)

    # ViT + GRU branch
    # NOTE: we already normalized images at load time, so DO NOT Rescale here
    x_vit = inputs  # use inputs directly
    patches = Patches(patch_size)(x_vit)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.2)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        encoded_patches = layers.Add()([x3, x2])

    x_vit = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    x_vit = layers.Dropout(0.002)(x_vit)
    x_vit = layers.Flatten()(x_vit)
    x_vit = layers.Reshape((-1, x_vit.shape[-1]))(x_vit)
    x_vit = layers.GRU(256)(x_vit)

    # Concatenate
    x = layers.concatenate([x_cnn, x_vit])
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.003)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

model = build_hybrid_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# -----------------------
# TRAIN (you can reduce epochs for debugging)
# -----------------------
start_time = time.time()

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,        # set to 100 when you are ready
                    batch_size=64)

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")

# -----------------------
# GRAD-CAM UTILITIES
# -----------------------
def find_last_conv_layer(model):
    # return name of last Conv2D layer
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
    return None

def make_gradcam_heatmap(model, img_array, last_conv_layer_name, class_index=None):
    """
    img_array: HxWxC, values in [0,1] as your model expects
    returns heatmap HxW (values 0..1)
    """
    img_tensor = tf.expand_dims(img_array, axis=0)  # 1,H,W,C

    last_conv_layer = model.get_layer(last_conv_layer_name)

    # build model mapping inputs -> conv outputs + predictions
    grad_model = tf.keras.models.Model([model.inputs], [last_conv_layer.output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        if class_index is None:
            class_index = tf.argmax(predictions[0])
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    # pooled grads across (h,w)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]  # H', W', channels
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-9)
    heatmap = heatmap.numpy()
    return heatmap

def overlay_gradcam_on_image(image, heatmap, alpha=0.4):
    """
    image: HxWx3 in [0,1]
    heatmap: HxW in [0,1]
    """
    heatmap_uint8 = np.uint8(255 * heatmap)
    jet = plt.cm.jet(heatmap_uint8 / 255.0)[:, :, :3]  # NxNx3 in [0,1]
    jet_resized = tf.image.resize(jet, (image.shape[0], image.shape[1])).numpy()
    overlay = image * (1 - alpha) + jet_resized * alpha
    overlay = np.clip(overlay, 0, 1)
    return overlay

# -----------------------
# CREATE AND SAVE GRADCAM EXAMPLES
# -----------------------
last_conv_name = find_last_conv_layer(model)
if last_conv_name is None:
    raise RuntimeError("No Conv2D layer found in model for Grad-CAM target.")

print("Using last conv layer for Grad-CAM:", last_conv_name)

# create output folder
out_dir = "gradcam_outputs"
os.makedirs(out_dir, exist_ok=True)

# pick some validation samples (change how many you want)
num_examples = 8 if X_val.shape[0] >= 8 else X_val.shape[0]
indices = np.random.choice(range(X_val.shape[0]), size=num_examples, replace=False)

for i, idx in enumerate(indices):
    img = X_val[idx]  # already in [0,1]
    true_label = y_val[idx]
    preds = model.predict(np.expand_dims(img, 0))
    pred_label = int(np.argmax(preds[0]))

    heatmap = make_gradcam_heatmap(model, img, last_conv_name, class_index=pred_label)
    overlay = overlay_gradcam_on_image(img, heatmap, alpha=0.45)

    # plot and save
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(img); axs[0].set_title(f"Input (true={true_label})"); axs[0].axis('off')
    axs[1].imshow(heatmap, cmap='jet'); axs[1].set_title(f"Heatmap (pred={pred_label})"); axs[1].axis('off')
    axs[2].imshow(overlay); axs[2].set_title("Overlay"); axs[2].axis('off')

    out_path = os.path.join(out_dir, f"gradcam_validx_{idx}_pred{pred_label}_true{true_label}.png")
    plt.tight_layout()
    fig.savefig(out_path, dpi=150)
    plt.close(fig)
    print(f"Saved Grad-CAM image to: {out_path}")

print("All Grad-CAM examples saved in", out_dir)


2025-11-25 05:54:18.055169: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764050058.331431      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764050058.406226      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

 Trainable params: 12,478,728 (47.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.6801 - loss: 0.8855 - val_accuracy: 0.9761 - val_loss: 0.0857
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.9820 - loss: 0.0617 - val_accuracy: 0.9847 - val_loss: 0.0578
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.9928 - loss: 0.0220 - val_accuracy: 0.9864 - val_loss: 0.0519
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.9957 - loss: 0.0092 - val_accuracy: 0.9915 - val_loss: 0.0474
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.9964 - loss: 0.0137 - val_accuracy: 0.9920 - val_loss: 0.0318
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - accuracy: 0.9982 - loss: 0.0063 - val_accuracy: 0.9943 - val_loss: 0.0195
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 217s 2s/step - accuracy: 0.9997 - loss: 0.0018 - val_accuracy: 0.9881 - val_loss: 0.0435
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.9975 - loss: 0.0080 - val_accu

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor']]
Received: inputs=Tensor(shape=(1, 128, 128, 3))
  warnings.warn(msg)


Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_143_pred6_true6.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_1148_pred1_true1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_438_pred6_true6.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_1068_pred2_true2.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_972_pred1_true1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_268_pred6_true6.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_1103_pred5_true5.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Saved Grad-CAM image to: gradcam_outputs/gradcam_validx_416_pred6_true6.png
All Grad-CAM examples saved in gradcam_outputs
